## Cluster Family Regressions

In this notebook, we mainly produce Figure 6 and the analogue figures in the SI.
We also compile the regression statistics table reported in the SI.

For all figures, we use the connected components of the Cluster Family Graph and name each of them by the largest single node contained in it.

### Preparations

In [ ]:
%run de_hue_order_alignment.py

from collections import defaultdict
import functools
from scipy import stats
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.legend import Legend
import seaborn as sns

from legal_data_clustering.utils.graph_api import cluster_families

sns.set_style('darkgrid')
plt.rcParams['axes.formatter.limits'] = (-10, 10) # for consistently non-scientific notation
plt.rcParams.update({'font.size': 14})

In [ ]:
def get_manual_labels(dataset):
    df = pd.read_csv(f'../graphics/labels-{dataset.lower()}.csv')
    return {cluster: label for cluster, label in zip(df['Cluster ID'], df['Label'])}

In [ ]:
    
def subtitle_decorator(handler):
    @functools.wraps(handler)
    def wrapper(legend, orig_handle, fontsize, handlebox):
        handle_marker = handler(legend, orig_handle, fontsize, handlebox)
        if handle_marker.get_alpha() == 0:
            handlebox.set_visible(False)
    return wrapper

# Adds our decorator to all legend handler functions
for handler in Legend.get_default_handler_map().values():
    handler.legend_artist = subtitle_decorator(handler.legend_artist)

In [ ]:
def filter_edges(G, threshold):
    edges_to_remove = [
        (u, v) 
        for u, v, data in G.edges(data=True) 
        if (
            data['weight'] < G.nodes[u]['weight'] * threshold or
            data['weight'] < G.nodes[v]['weight'] * threshold 
        )
    ]
    H = G.copy()
    H.remove_edges_from(edges_to_remove)
    return H


def count_weight_per_year(nodes_set, H):
    counts = defaultdict(int)
    for node in nodes_set:
        year = node.split('_')[0]
        counts[year] += H.nodes[node]['weight']
    return dict(counts)


def get_connected_components_per_year(H):
    components = list(nx.connected_components(H.to_undirected()))
    components.sort(
        key=lambda nodes_set: (max([H.nodes[n]["tokens_n"] for n in nodes_set]), sorted(nodes_set)[-1]),
        reverse=True
    )
    component_weight_per_year = [
        count_weight_per_year(nodes_set, H)
        for nodes_set in components
    ]
    return component_weight_per_year, components

def get_linear_regression_df(df):
    linreg = [(
        label, 
        stats.linregress(range(len(row)), row.fillna(0).to_list())
    ) for label, row in df.T.iterrows()]
    
    number_of_years = len(df.T.columns)

    linreg_df = pd.DataFrame([
        dict(
            label=label, 
            intercept=d.intercept, 
            slope=d.slope, 
            rvalue=d.rvalue, 
            r2value=d.rvalue**2, 
            pvalue=d.pvalue, 
            stderr=d.stderr
        ) 
        for label, d in linreg
    ])
    linreg_df = linreg_df.set_index('label')
    pd.set_option('display.float_format', lambda x: '%.2f' % x)
    linreg_df['size_mean'] =  df.fillna(0).mean()
    linreg_df['slope_rel'] = linreg_df.slope / linreg_df.size_mean
    linreg_df['last_year_value'] = linreg_df.intercept + linreg_df.slope * (number_of_years - 1)
    return linreg_df

In [ ]:
def format_func(value, tick_number):
        if value == 0:
            return '0.0'
        return f'{value:.1e}'.replace('e+0', 'e')
    

def plot_community_development(
    config_str, dataset, threshold, name, 
    selected_clusters=10, colors=None, 
    most_growing_labels=None, hue_order=None,
    show_manual_labels=True
):
    # Load Graph
    global G
    G = nx.read_gpickle(
        path=f'../../legal-networks-data/{dataset.lower()}/13_cluster_evolution_graph/all_{config_str}.gpickle.gz',
    )

    # Select weights
    nx.set_node_attributes(G, nx.get_node_attributes(G, 'tokens_n'), 'weight')
    nx.set_edge_attributes(G, nx.get_edge_attributes(G, 'tokens_n'), 'weight')
    
    H = filter_edges(G, threshold)
    

    component_weight_per_year, components = get_connected_components_per_year(H)

    # Format numbers
    df = pd.DataFrame(component_weight_per_year).T.sort_index()
    df.columns = [
        sorted(nodes_set, key=lambda n: (H.nodes[n]["tokens_n"], n))[-1]
        for nodes_set in components
    ]
    
    # Setup cluster filter
    if type(selected_clusters) is int:
        selected_clusters = df.columns[:selected_clusters]
        selected_cluster_labels = None
    elif type(selected_clusters) is dict:
        selected_cluster_labels = selected_clusters
        selected_clusters = list(selected_clusters.keys())
    elif type(selected_clusters) is list:
        selected_cluster_labels = None
    else:
        raise Exception('Wrong selected_clusters type')
    try:
        df_selected = df.loc[:,selected_clusters]
    except KeyError :
        print('Missing:', sorted(set(selected_clusters) - set(df.columns)))
        raise
        
    # Calculate growth of selected clusters
    growth_selected = df_selected.iloc[-1].sum() - df_selected.iloc[0].sum()
    print('Selected growth (abs.):', growth_selected)
    growth_total = df.iloc[-1].sum() - df.iloc[0].sum()
    print('Total growth (abs.):', growth_total)
    print('Selected growth (rel.):', growth_selected/growth_total)
    
    selected_size_ratio_df = df_selected.T.sum() / df.T.sum()
    print('Selected sizes per year statistics:')
    print(selected_size_ratio_df.describe())
    
    # Convert data into long format (for seaborn)
    global df_melt
    df_melt = pd.melt(df_selected.reset_index(), id_vars=['index'])
    df_melt.columns = ['Year', 'Cluster', 'Tokens']
    
    # Hue order
    leading_clusters = [c[0] for c in cluster_families(G, 0.15)]
    if not hue_order:
        hue_order = sorted(
            df_melt.Cluster.unique(),
            key=lambda x: leading_clusters.index(x)
        )

    # Plot scatter
    plt.figure(figsize=(9, 6))
    ax = sns.scatterplot(
        x='Year', 
        y='Tokens', 
        hue='Cluster',
        hue_order=hue_order,
        data=df_melt, 
        palette=cluster_family_colors(dataset) if colors is None else colors,
    )
    ax.set_xticklabels(df.T.columns, rotation=90)
    
    # Regression
    linreg_df = get_linear_regression_df(df)
    line_df = pd.DataFrame([
        {
            'label': label,
            df.T.columns[0]: row.intercept,
            df.T.columns[-1]: row.intercept + row.slope * (len(df.T.columns) - 1)
        }
        for label, row in linreg_df.iterrows()
    ])
    line_df = line_df.set_index('label')
    
    line_df_selected = line_df.loc[selected_clusters]
    line_df_selected.index.names = ['label']
    
    # Convert data into long format (for seaborn)
    df_line_melt = pd.melt(line_df_selected.reset_index(), id_vars=['label'])
    df_line_melt.columns = ['Cluster', 'Year', 'Tokens']
        
    # Plot lines
    sns.lineplot(
        x='Year', 
        y='Tokens', 
        hue='Cluster', 
        hue_order=hue_order,
        data=df_line_melt, 
        palette=cluster_family_colors(dataset) if colors is None else colors,
        legend=False,
    )
        
    handles, labels = ax.get_legend_handles_labels()
    handle_dict = {label: handle for handle, label in zip(handles, labels)}
    labels = line_df_selected.sort_values(line_df_selected.columns[-1], ascending=False).index.to_list()
    handles = [handle_dict[l] for l in labels]

    manual_labels = get_manual_labels(dataset)
    labels = [f'{l.split("_")[-1]} in {l.split("_")[0][:4]}' for l in labels]
    if show_manual_labels:
        labels = [
            f'{manual_labels[l]} – {l}' if l in manual_labels else l
            for l in labels
        ]
    
    legend = ax.legend(handles, labels, loc='upper center', 
                bbox_to_anchor=(0.5, -0.15), frameon=False, ncol=(1 if show_manual_labels else 4))
    ax.set_xticklabels(range(1994,2019))
    plt.ylim(0,3000000 if dataset == 'us' else 900000) 

    ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
    
    plt.savefig(
        f'../graphics/cluster-evolution-size-dynamics-{dataset.lower()}-{config_str}-{name}.pdf', 
        bbox_extra_artists=(legend,), 
        bbox_inches='tight'
    )
    
    linreg_df_selected = linreg_df.loc[selected_clusters]
    return linreg_df_selected, components, hue_order

In [ ]:
def analyse_slope_to_size(linreg_df_top, dataset, name):
    plt.figure(figsize=(9, 6))
    regplt = sns.regplot(linreg_df_top.slope, linreg_df_top.size_mean)
    regplt.set(
        ylim=(0, None),
        xlabel='Slope',
        ylabel='Mean size'
    )
    regplt.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
    regplt.yaxis.set_major_formatter(plt.FuncFormatter(format_func))


    print(stats.linregress(
        linreg_df_top.slope.fillna(0), 
        linreg_df_top.size_mean.fillna(0),
    ))
    plt.savefig(
        f'../graphics/cluster-evolution-slope-size-regression-{dataset.lower()}-{name}.pdf', 
    )
    

def analyse_slope_rel_to_size(linreg_df_top):
    ax = sns.regplot(linreg_df_top.slope_rel, linreg_df_top.size_mean)
    print(stats.linregress(
        linreg_df_top.slope_rel.fillna(0), 
        linreg_df_top.size_mean.fillna(0)
    ))

### US regression plots

#### Top 20 for inspection

In [ ]:
linreg_df_top_us, components, hue_order_us = plot_community_development(
    config_str='0-0_1-0_-1_a-infomap_n100_m1-0_s0_c1000', 
    dataset='us', 
    threshold=0.15,
    name='top20',
    selected_clusters=20
)

In [ ]:
analyse_slope_to_size(linreg_df_top_us, 'us', 'top20')

In [ ]:
linreg_df_top_us

#### Selected for print

In [ ]:
linreg_df_top, components, _ = plot_community_development(
    config_str='0-0_1-0_-1_a-infomap_n100_m1-0_s0_c1000', 
    dataset='us', 
    threshold=0.15,
    name='selected',
    selected_clusters={
        # Growing
        '2015_3': 'Public health and social welfare', 
        '2010_0': 'Financial regulation',
        '2011_3': 'Student loans and economic support',
        '2015_11': 'Tax',
        '2015_4': 'Foreign assistance',
        # Stagnating
        '2009_8': 'Public housing',
        '1995_3': 'Agriculture',
    },
    most_growing_labels=['2015_3', '2010_0', '2011_3', '2015_11', '2015_4'],
    hue_order=hue_order_us,
)

In [ ]:
analyse_slope_to_size(linreg_df_top, 'us', 'selected')

### DE regression plots

#### Top 20 for inspection

In [ ]:
linreg_df_top_de, components, hue_order_de = plot_community_development(
    config_str='0-0_1-0_-1_a-infomap_n100_m1-0_s0_c1000', 
    dataset='de', 
    threshold=0.15,
    selected_clusters=20,
    name='top20'
)

In [ ]:
analyse_slope_to_size(linreg_df_top_de, 'de', 'top20')

In [ ]:
linreg_df_top_de.sort_values('slope', ascending=False)

In [ ]:
linreg_df_top_de.sort_values('slope')[['slope']]

#### Selected for print

In [ ]:
linreg_df_top, components, _ = plot_community_development(
    config_str='0-0_1-0_-1_a-infomap_n100_m1-0_s0_c1000', 
    dataset='de', 
    threshold=0.15,
    name='selected',
    selected_clusters={
        # Growing
        '2018-01-01_2': 'Social welfare',
        '2017-01-01_8': 'Financial regulation',
        '2018-01-01_34': 'Renewable energy',
        '2018-01-01_0': 'Tax',
        '2015-01-01_15': 'Corporations',
        # Stagnating
        '2000-01-01_16': 'Reparations',
        '2011-01-01_21': 'Inheritance',
    },
    most_growing_labels=['2018-01-01_2', '2017-01-01_8', '2018-01-01_34', '2018-01-01_0', '2015-01-01_15'],
    hue_order=hue_order_de
)

In [ ]:
analyse_slope_to_size(linreg_df_top, 'de', 'selected')

In [ ]:
linreg_df_top.sort_values('slope')

### Table with regression values

In [ ]:
linreg_df_top_us.sort_values('last_year_value', ascending=False)

In [ ]:
def save_linreg_df(df, country_code):
    df = df.sort_values('last_year_value', ascending=False)
    df.index = [f'{l.split("_")[-1]} in {l.split("_")[0][:4]}' for l in df.index]
    df = df.reset_index()
    df = df[[
        'index', 
        'slope', 
        'intercept', 
        'pvalue', 
        'rvalue',  
        'r2value',  
        'stderr', 
        'size_mean'
    ]]
    df.columns = [
        'Cluster\\\\family', 
        'Slope', 'Intercept', 
        'P-value', 
        'Correlation\\\\coefficient ($r$)', 
        '$r^2$', 
        'Standard\\\\error', 
        'Mean value'
    ]
    df['Country\\\\code'] = country_code
    df = df.set_index(['Country\\\\code', 'Cluster\\\\family'])
    return df

latex_df = pd.concat([
    save_linreg_df(linreg_df_top_us, 'US'),
    save_linreg_df(linreg_df_top_de, 'DE')
]).sort_values(['Country\\\\code', 'Slope'], ascending=False)
cols =  [
    '\multicolumn{1}{p{14mm}}{\centering ' + c + '}' 
    for c in latex_df.columns
]
cols[0] = cols[0].replace('14mm', '18mm')
cols[1] = cols[1].replace('14mm', '18mm')
cols[2] = cols[2].replace('14mm', '10mm')
cols[3] = cols[3].replace('14mm', '16mm')
cols[4] = cols[4].replace('14mm', '10mm')
cols[5] = cols[5].replace('14mm', '16mm')
cols[6] = cols[6].replace('14mm', '18mm')


latex_df.columns = cols

latex_df.index.names = [
    '\multicolumn{1}{p{14mm}}{\centering ' + c + '}' 
    for c in latex_df.index.names
]

with open(f'../graphics/cluster_family_growth_stats.tex', 'w') as f:
    latex_df.to_latex(f, escape=False, index=True)

In [ ]:
latex_df

### Inspect other configs

In [ ]:
for dataset in ['us', 'de']:
    for config_str in ['0-0_1-0_-1_a-infomap_m1-0_s0_c1000'] + [
        f'0-0_1-0_-1_a-infomap_n{runs}_m1-0_s0_c1000' for runs in list(range(60,150+1,10)) + [200]
    ]:
        print('starting', config_str)
        if config_str != f'0-0_1-0_-1_a-infomap_n100_m1-0_s0_c1000':
            # plotted already above
            plot_community_development(
                config_str, 
                dataset, 
                threshold=0.15,
                name='top20',
                selected_clusters=20,
                show_manual_labels=False
            )

### End

